<a href="https://colab.research.google.com/github/dannytsao/DannyTsao/blob/master/JJCSV2EXCL_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from google.colab import drive


# Mount Google Drive
drive.mount('/content/drive')

# Authenticate with Google Drive
drive_path = '/content/drive/MyDrive'

# Authenticate with Google Sheets
scope = ['https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/spreadsheets']

credentials = ServiceAccountCredentials.from_json_keyfile_name('/content/drive/MyDrive/JJVC_WIP/jjvc-392508-765246081d65.json', scope)
gc = gspread.authorize(credentials)


# Open the Google Sheets file
sheet = gc.open('JJVC_OTM')
worksheet = sheet.get_worksheet(0)


def copy_csv_to_excel(csv_file_path, excel_file_path, output_excel_file_path):
    try:
        # Read data from the Google Sheets file
        data = worksheet.get_all_values()
        df_csv = pd.DataFrame(data[1:], columns=data[0])

        # Read the existing Excel file
        df_excel = pd.read_excel(excel_file_path, sheet_name='ORDER')

        # Modify the df_excel DataFrame with desired changes
        df_excel['OR_CLIENTREFERENCENO'] = df_csv['Delivery']
        df_excel['OR_ORDERSOURCEID'] = df_csv['Delivery']
        df_excel['DST_LOCATIONNAME'] = df_csv['Ship-to party']
        df_excel['DST_CONTACTNO'] = df_csv['Telephone']
        df_excel['DST_ADDRESS1'] = df_csv['Street']
        df_excel['ORL_PCSQTY'] = df_csv['nit']
        df_excel['OR_ORDERADDDATE'] = df_csv['Created on']
        df_excel['OR_ORDERADDDATE'] = pd.to_datetime(df_excel['OR_ORDERADDDATE'])
        df_excel['OR_REQDELIVERYDATE'] = df_excel['OR_ORDERADDDATE'] + pd.DateOffset(days=1)

        # Set default values in the respective columns
        default_values = {
            'OR_COUNTRY': 'TWN',
            'OR_PRINCIPAL': 'JNJ',
            # Rest of the default values
        }

        for column, default_value in default_values.items():
            df_excel[column] = default_value

        # Save the modified Excel file
        writer = pd.ExcelWriter(output_excel_file_path, engine='xlsxwriter')
        df_excel.to_excel(writer, index=False, sheet_name='ORDER')
        writer.save()
        writer.close()

        print(f"Data copied from Google Sheets to {output_excel_file_path} successfully.")

    except gspread.exceptions.APIError as e:
        print("An API error occurred:", e)
    except Exception as e:
        print("An error occurred:", e)

# Define file paths
# Define file paths
csv_file_path = drive_path + '/JJVC_WIP/JJVC_OTM.gsheet'
excel_file_path = drive_path + '/JJVC_WIP/ORDER_TEMPLATE_JNJ.gsheet'
output_excel_file_path = drive_path + '/JJVC_WIP/ORDER_WIP.gsheet'

# Copy data from CSV to Excel and set default value in 'OR_COUNTRY' column
copy_csv_to_excel(csv_file_path, excel_file_path, output_excel_file_path)


Mounted at /content/drive
An error occurred: [Errno 2] No such file or directory: '/content/drive/MyDrive/JJVC_WIP/ORDER_TEMPLATE_JNJ.gsheet'
